In [2]:
# !pip install transformers --quiet
# !pip install torch torchvision torchaudio
# !pip install opencv-python
# !pip install torchvggish
# !pip install soundfile

In [7]:
from transformers import AutoImageProcessor, ResNetModel
import torch
import numpy as np

In [8]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetModel.from_pretrained("microsoft/resnet-50")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at microsoft/resnet-50 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import cv2
import glob
print(cv2.__version__)

4.7.0


In [24]:

ethnicities = ["british", "chinese", "german", "greek", "hungarian", "serbian"] 

for ethnicity in ethnicities:
    i = 1

    root = './data/'+ethnicity+'/SEWA DB v0.2/'
    save_root = './data/'+ethnicity+'/avi/'
    for filepath in glob.iglob(root + '*/*.avi'):
        print(i, filepath)  
        vidcap = cv2.VideoCapture(filepath)
        success, image = vidcap.read()

        inputs = image_processor(image, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**inputs)

        last_hidden_state_arr = outputs.pooler_output
        # print(outputs.pooler_output.shape)
        count = 1

        while success:
            vidcap.set(cv2.CAP_PROP_POS_MSEC, (count*1000))
            success, image = vidcap.read()

            if success:
                # HuggingFace
                inputs = image_processor(image, return_tensors="pt")

                with torch.no_grad():
                    outputs = model(**inputs)

                last_hidden_state_arr = torch.vstack((last_hidden_state_arr, outputs.pooler_output))
                count = count + 1

        # avg_last_hidden_state /= count
        i+=1
        print(last_hidden_state_arr.shape)
        torch.save(last_hidden_state_arr, save_root + filepath.split('/')[-1].split('.')[0] + '.pt')

1 ./data/british/SEWA DB v0.2/C4_S095_P189_AD1/Round1_SessionId_383_VideoId_1_UserId_63458cbe-4561-4944-8f56-75545c214e2c.avi
torch.Size([62, 2048, 1, 1])
2 ./data/british/SEWA DB v0.2/C4_S095_P189_AD2/Round1_SessionId_383_VideoId_2_UserId_63458cbe-4561-4944-8f56-75545c214e2c.avi
torch.Size([33, 2048, 1, 1])
3 ./data/british/SEWA DB v0.2/C4_S095_P189_AD3/Round1_SessionId_383_VideoId_3_UserId_63458cbe-4561-4944-8f56-75545c214e2c.avi
torch.Size([49, 2048, 1, 1])
4 ./data/british/SEWA DB v0.2/C4_S095_P190_AD1/Round1_SessionId_383_VideoId_1_UserId_a67cf20f-43e1-4279-ab33-2a31470e6086.avi
torch.Size([62, 2048, 1, 1])
5 ./data/british/SEWA DB v0.2/C4_S095_P190_AD2/Round1_SessionId_383_VideoId_2_UserId_a67cf20f-43e1-4279-ab33-2a31470e6086.avi
torch.Size([34, 2048, 1, 1])
6 ./data/british/SEWA DB v0.2/C4_S095_P190_AD3/Round1_SessionId_383_VideoId_3_UserId_a67cf20f-43e1-4279-ab33-2a31470e6086.avi
torch.Size([51, 2048, 1, 1])
7 ./data/british/SEWA DB v0.2/C4_S096_P191_AD1/Round1_SessionId_39_Vid

In [ ]:
print(avg_last_hidden_state.shape)

In [5]:
from torchvggish import vggish, vggish_input

In [11]:
embedding_model = vggish()
embedding_model.eval()

ethnicity = "british"

save_root = './data/'+ethnicity+'/wav/'

i = 1
root = './data/'+ethnicity+'/SEWA DB v0.2/'
for filepath in glob.iglob(root + '*/*.wav'):
    print(i, filepath)
    example = vggish_input.wavfile_to_examples(filepath)
    embeddings = embedding_model.forward(example)
    torch.save(embeddings, save_root + filepath.split('/')[-1].split('.')[0] + '.pt')
    i+=1

1 ./data/british/SEWA DB v0.2/C4_S095_P189_AD1/Round1_SessionId_383_VideoId_1_UserId_63458cbe-4561-4944-8f56-75545c214e2c.wav
2 ./data/british/SEWA DB v0.2/C4_S095_P189_AD2/Round1_SessionId_383_VideoId_2_UserId_63458cbe-4561-4944-8f56-75545c214e2c.wav
3 ./data/british/SEWA DB v0.2/C4_S095_P189_AD3/Round1_SessionId_383_VideoId_3_UserId_63458cbe-4561-4944-8f56-75545c214e2c.wav
4 ./data/british/SEWA DB v0.2/C4_S095_P190_AD1/Round1_SessionId_383_VideoId_1_UserId_a67cf20f-43e1-4279-ab33-2a31470e6086.wav
5 ./data/british/SEWA DB v0.2/C4_S095_P190_AD2/Round1_SessionId_383_VideoId_2_UserId_a67cf20f-43e1-4279-ab33-2a31470e6086.wav
6 ./data/british/SEWA DB v0.2/C4_S095_P190_AD3/Round1_SessionId_383_VideoId_3_UserId_a67cf20f-43e1-4279-ab33-2a31470e6086.wav
7 ./data/british/SEWA DB v0.2/C4_S096_P191_AD1/Round1_SessionId_39_VideoId_1_UserId_6bb55080-9de9-4d1c-a834-45a417e17058.wav
8 ./data/british/SEWA DB v0.2/C4_S096_P191_AD2/Round1_SessionId_39_VideoId_2_UserId_6bb55080-9de9-4d1c-a834-45a417e1705

In [12]:
import json
import glob

def parse_questionnaire(path):
    with open(path, 'r') as j:
        contents = json.loads(j.read())
    return (int(contents[0]['value']), 
            int(contents[1]['value']), 
            int(contents[2]['value']), 
            int(contents[3]['value']), 
            int(contents[4]['value']))

In [13]:
i = 1
wavs = []
avis = []
values = []
ethincity = []
root = './data'
for filepath in glob.iglob(root + '/*/wav/*.pt'):
    ethnic = filepath.split('/')[2]
    wavs.append(ethnic + '/wav/' + filepath.split('/')[-1])
for filepath in glob.iglob(root + '/*/avi/*.pt'):
    ethnic = filepath.split('/')[2]
    ethincity.append(ethnic)
    avis.append(ethnic + '/avi/' + filepath.split('/')[-1])
for filepath in glob.iglob(root + '/*/SEWA*/*/*questionnaire.json'):
    value = parse_questionnaire(filepath)
    values.append(value)

In [14]:
i = 1
with open('key.csv', 'w') as f:
    f.write('id,video,audio,good,like,excite,positive,negative,ethnicity\n')
    for a, b, c, d in zip(avis, wavs, values, ethincity):
        f.write("{},{},{},{},{},{},{},{},{}\n".format(i, a, b, c[0], c[1], c[2], c[3], c[4], d))
        i+=1